In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import sqlalchemy as db

In [2]:
con = db.create_engine("sqlite:///C:/Users/zheji/Desktop/TradingTools/fundamentalDB.sqlite")

In [ ]:
print(con.table_names())
con.execute("DROP TABLE insider")

# Check Insider Data

In [3]:
df = pd.read_sql('insider', con)
print(df)

      index ticker  trans_date     price  num_shares direction trans_type  \
0       0.0   MSFT  2020-06-30    0.0000        4000         D          G   
1       1.0   MSFT  2020-06-04    0.0000         273         A          A   
2       2.0   MSFT  2020-06-04    0.0000         273         A          A   
3       3.0   MSFT  2020-06-04  182.9200          82         D          F   
4       4.0   MSFT  2020-06-08  186.0513          49         D          S   
...     ...    ...         ...       ...         ...       ...        ...   
2433    NaN     FB  2020-02-15    0.0000       10023         A          M   
2434    NaN     FB  2020-02-15    0.0000        8254         A          M   
2435    NaN     FB  2020-02-15    0.0000        8432         A          M   
2436    NaN     FB  2020-02-15    0.0000        9426         A          M   
2437    NaN     FB  2020-02-15  214.1800       30539         D          F   

                         name                      position isdirector  \
0

In [4]:
df[df['ticker'] == 'FB']

,index,ticker,trans_date,price,num_shares,direction,trans_type,name,position,isdirector,isofficer,ismajor,isother,owned_shares
2339,NaN,FB,2020-07-21,246.22,194,D,S,Newstead Jennifer,VP and General Counsel,0,1,0,0,2112
2340,NaN,FB,2020-07-14,236.76,194,D,S,Newstead Jennifer,VP and General Counsel,0,1,0,0,2306
2341,NaN,FB,2020-07-07,239.64,194,D,S,Newstead Jennifer,VP and General Counsel,0,1,0,0,2500
2342,NaN,FB,2020-06-30,220.60,194,D,S,Newstead Jennifer,VP and General Counsel,0,1,0,0,2694
2343,NaN,FB,2020-06-23,241.36,194,D,S,Newstead Jennifer,VP and General Counsel,0,1,0,0,2888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,NaN,FB,2020-02-15,0.00,10023,A,M,Schroepfer Michael Todd,Chief Technology Officer,0,1,0,0,449117
2434,NaN,FB,2020-02-15,0.00,8254,A,M,Schroepfer Michael Todd,Chief Technology Officer,0,1,0,0,457371
2435,NaN,FB,2020-02-15,0.00,8432,A,M,Schroepfer Michael Todd,Chief Technology Officer,0,1,0,0,465803
2436,NaN,FB,2020-02-15,0.00,9426,A,M,Schroepfer Michael Todd,Chief Technology Officer,0,1,0,0,475229


In [33]:
df[df['trans_type'] == 'A']

,index,ticker,trans_date,price,num_shares,direction,trans_type,name,position,isdirector,isofficer,ismajor,isother,owned_shares
1,1,MSFT,2020-06-04,0.0,273,A,A,WARRIOR PADMASREE,,1,0,0,0,10998
2,2,MSFT,2020-06-04,0.0,273,A,A,Walmsley Emma N,,1,0,0,0,424
7,8,MSFT,2020-06-04,0.0,273,A,A,STANTON JOHN W,,1,0,0,0,74856
8,9,MSFT,2020-06-04,0.0,273,A,A,SORENSON ARNE M,,1,0,0,0,4093
9,10,MSFT,2020-06-04,0.0,273,A,A,SCHARF CHARLES W,,1,0,0,0,40843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,268,MSFT,2018-01-31,0.0,526,A,A,STANTON JOHN W,,1,0,0,0,71289
266,269,MSFT,2018-01-31,0.0,526,A,A,SORENSON ARNE M,,1,0,0,0,526
267,270,MSFT,2018-01-31,0.0,526,A,A,SCHARF CHARLES W,,1,0,0,0,37276
268,271,MSFT,2018-01-31,0.0,526,A,A,PANKE HELMUT,,1,0,0,0,52232


# Historical Filing

In [6]:
def get_historical_links(stock):
    start = 0
    links = []
    while start % 100 == 0:
        url = "https://www.sec.gov/cgi-bin/browse-edgar?CIK={}&owner=include&action=getcompany&type=4&count=100&start={}".format(stock.lower(), start)
        html = urlopen(url)
        bsObj = BeautifulSoup(html.read())
        nameList = bsObj.findAll('a')
        for link in nameList:
            sep = link.get('href').split('/')
            if 'Archives' in sep and 'data' in sep:
                links.append(link.get('href'))
                start = start + 1
    print(len(links))
    return links

In [6]:
links = get_historical_links('W')

1621


In [9]:
xml_reports_link = find_all_xml(links[:50])

Getting  /Archives/edgar/data/1616707/000161670720000128/0001616707-20-000128-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000126/0001616707-20-000126-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000124/0001616707-20-000124-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000122/0001616707-20-000122-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000120/0001616707-20-000120-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000118/0001616707-20-000118-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000116/0001616707-20-000116-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000114/0001616707-20-000114-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000112/0001616707-20-000112-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000110/0001616707-20-000110-index.htm
Getting  /Archives/edgar/data/1616707/000161670720000108/0001616707-20-000108-index.htm
Getting  /Archives/edgar/data/16

In [24]:
df = get_filing_data(xml_reports_link)

Erorr Finding Owner Type
Erorr Finding Owner Type
Erorr Finding Owner Type
Erorr Finding Owner Type
Erorr Finding Owner Type
Erorr Finding Owner Type
(143, 10)
(143, 10)


In [25]:
pd.options.display.max_rows = 100
df[df['direction']=='D']

,ticker,trans_date,price,num_shares,direction,trans_type,name,position,person_type,owned_shares
1,W,2020-07-16,214.26,485,D,S,Macri Edmond,Chief Product/Marketing,officer,2515
2,W,2020-07-16,215.26,453,D,S,Macri Edmond,Chief Product/Marketing,officer,2062
3,W,2020-07-16,216.22,411,D,S,Macri Edmond,Chief Product/Marketing,officer,1651
5,W,2020-07-16,214.08,267,D,S,Oblak Steve,Chief Merchandising Officer,officer,147540
6,W,2020-07-16,214.85,805,D,S,Oblak Steve,Chief Merchandising Officer,officer,146735
7,W,2020-07-16,216.41,278,D,S,Oblak Steve,Chief Merchandising Officer,officer,146457
8,W,2020-07-17,222.75,825,D,S,Oblak Steve,Chief Merchandising Officer,officer,145632
10,W,2020-07-16,213.84,632,D,S,FLEISHER MICHAEL D,Chief Financial Officer,officer,112042
11,W,2020-07-16,214.43,2469,D,S,FLEISHER MICHAEL D,Chief Financial Officer,officer,109573
12,W,2020-07-16,215.49,2281,D,S,FLEISHER MICHAEL D,Chief Financial Officer,officer,107292


# Today Filing Home Page

In [7]:
html = urlopen("https://www.sec.gov/cgi-bin/current?q1=0&q2=0&q3=4") 

In [8]:
bsObj = BeautifulSoup(html.read())
bsObj

<html>
<head>
<title>EDGAR Current Events</title>
<script language="JavaScript" src="/include/sec.js" type="text/javascript"></script>
</head>
<body bgcolor="#FFFFFF" link="#807331" vlink="#FF0000">
<!-- SEC Web Analytics - For information please visit: http://www.sec.gov/privacy.htm#collectedinfo -->
<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-TD3BKV');</script>
<!-- End SEC Web Analytics -->
<br/>
<table width="100%"><tr><td width="5%"></td>
<td width="80%">
<h1>Results for EDGAR Current Events</h1>
<p>The total number of matches for 2020-07-27 is <

In [9]:
def find_all_reports_today():
    html = urlopen("https://www.sec.gov/cgi-bin/current?q1=0&q2=0&q3=4") 
    bsObj = BeautifulSoup(html.read())
    nameList = bsObj.findAll('a')
    reports_link = []
    for link in nameList:
        sep = link.get('href').split('/')
#         print(link)
#         print(link.text)
        if link.text in ['4', '4/A']:
            reports_link.append(link.get('href'))
    return reports_link
reports_web_link = find_all_reports_today()
len(reports_web_link)

662

In [10]:
def find_all_xml(reports_web_link):
    xml_reports_link = []
    for link in reports_web_link:
        print("Getting ", link)
        url = 'https://sec.gov'+link
        html = urlopen(url)
        bsObj = BeautifulSoup(html.read())
        nameList = bsObj.findAll('a') 
        for link in nameList:
            filename = link.text
            if 'xml' in filename.split('.'):
                #print(link.get('href'))
                xml_reports_link.append(link.get('href'))
    return xml_reports_link
xml_reports_link = find_all_xml(reports_web_link[:20])

Getting  /Archives/edgar/data/1023731/0001023731-20-000090-index.html
Getting  /Archives/edgar/data/1023731/0001023731-20-000092-index.html
Getting  /Archives/edgar/data/1023731/0001023731-20-000094-index.html
Getting  /Archives/edgar/data/1023731/0001023731-20-000096-index.html
Getting  /Archives/edgar/data/1023731/0001023731-20-000098-index.html
Getting  /Archives/edgar/data/1800/0001179110-20-008493-index.html
Getting  /Archives/edgar/data/1354040/0001209191-20-043703-index.html
Getting  /Archives/edgar/data/1604028/0000899243-20-020484-index.html
Getting  /Archives/edgar/data/1604028/0000899243-20-020485-index.html
Getting  /Archives/edgar/data/1604028/0000899243-20-020486-index.html
Getting  /Archives/edgar/data/1604028/0000899243-20-020487-index.html
Getting  /Archives/edgar/data/1604028/0000899243-20-020488-index.html
Getting  /Archives/edgar/data/1604028/0000899243-20-020489-index.html
Getting  /Archives/edgar/data/1604028/0000899243-20-020490-index.html
Getting  /Archives/edga

In [11]:
def get_filing_data(xml_reports_link):
    all_trans = []
    ct = 0
    for xml in xml_reports_link:
        url = "https://sec.gov" + xml
        html = urlopen(url)
        bsObj = BeautifulSoup(html.read())
        symbol = bsObj.issuertradingsymbol.text
        name = bsObj.reportingowner.reportingownerid.rptownername.text
        owner = bsObj.reportingownerrelationship
        # person_type_map = {'0010': 'major holder',
        #                    '0100': 'officer',
        #                    '1000': 'director',
        #                    '0001': 'other',
        #                    'unknown': None}
        try:
            title = owner.officertitle.text
        except:
            print("Erorr Finding Insider Position")
            title = ''
        stock_trans = bsObj.findAll('nonderivativetransaction')
        if len(stock_trans) == 0:
        	print("No stock transactions")
        	continue
        for trans in stock_trans:
            try:
                trans_type = trans.transactioncode.text
            except:
                print("Error Finding Transaction Type")
                trans_type = None
            try:
                all_trans.append({'ticker':symbol, 
                                  'trans_date':trans.transactiondate.value.string,
                                  'price':float(trans.transactionpricepershare.value.string),
                                  'num_shares':int(float(trans.transactionshares.value.string)),
                                  'direction':trans.transactionacquireddisposedcode.value.string,
                                  'trans_type': trans_type,
                                  'name':name,
                                  'position':title,
                                  'isdirector': owner.isdirector.text,
                                  'isofficer': owner.isofficer.text,
                                  'ismajor':owner.istenpercentowner.text,
                                  'isother': owner.isother.text,
                                  'owned_shares': int(float(trans.sharesownedfollowingtransaction.value.string)),})
                ct = ct + 1
            except Exception as e:
                print("Error {} occured for getting filing data for {}".format(e, symbol))
                print(url)
    print("Total Number of transactions ", ct)
    df = pd.DataFrame(all_trans)
    print(df.shape)
    df = df.drop_duplicates()
    print(df.shape)
    return df

In [12]:
df = get_filing_data(xml_reports_link)

Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Erorr Finding Insider Position
Total Number of transactions  25
(25, 13)
(25, 13)


In [22]:
def get_filing_data(xml_reports_link):
    all_trans = []
    for xml in xml_reports_link:
        url = "https://sec.gov" + xml
        html = urlopen(url)
        bsObj = BeautifulSoup(html.read())
        #print(bsObj)
        symbol = bsObj.issuertradingsymbol.text
        print("URL: ", url)
        print("Getting info for ", symbol)
        name = bsObj.reportingowner.reportingownerid.rptownername.text
        owner = bsObj.reportingownerrelationship
#         person_type_map = {'0010': 'major holder',
#                            '0100': 'officer',
#                            '1000': 'director',
#                            '0001': 'other',
#                            'unknown': None}
#         try:
#             person_type = owner.isdirector.text + owner.isofficer.text\
#                 + owner.istenpercentowner.text +owner.isother.text
#             title = owner.officertitle.text
#             person_type = person_type_map[person_type]
#         except:
#             print("Erorr Finding Owner Type")
#             person_type = 'unknown'
        for trans in bsObj.findAll('nonderivativetransaction'):
            try:
                trans_type = trans.transactioncode.text
            except:
                print("Error Finding Transaction Type")
                trans_type = None
            print(owner.officertitle.text)
            print(owner.isdirector.text)
            print(owner.isofficer.text)
            print(owner.istenpercentowner.text)
            print(owner.isother.text)
            try:
                all_trans.append({'ticker':symbol, 
                                  'trans_date':trans.transactiondate.value.string,
                                  'price':float(trans.transactionpricepershare.value.string),
                                  'num_shares':int(float(trans.transactionshares.value.string)),
                                  'direction':trans.transactionacquireddisposedcode.value.string,
                                  'trans_type': trans_type,
                                  'name':name,
                                  'position':owner.officertitle.text,
                                  'isdirector': owner.isdirector.text,
                                  'isofficer': owner.isofficer.text,
                                  'ismajor':owner.istenpercentowner.text,
                                  'isother': owner.isother.text,
                                  'owned_shares': int(float(trans.sharesownedfollowingtransaction.value.string)),})
            except Exception as e:
                print("Error {} occured for getting filing data for {}".format(e, symbol))
                print(url)
    df = pd.DataFrame(all_trans)
    print(df.shape)
    df = df.drop_duplicates()
    print(df.shape)
    return df
df = get_filing_data(xml_reports_link)


https://sec.gov/Archives/edgar/data/1047340/000104734020000217/wf-form4_159562888566654.xml
Getting info for  FDP
Chairman and CEO
1
1
1
0
https://sec.gov/Archives/edgar/data/318299/000128073120000003/primary_doc.xml
Getting info for  SRCO
https://sec.gov/Archives/edgar/data/863821/000110465920086571/a4.xml
Getting info for  RH


AttributeError: 'NoneType' object has no attribute 'text'

In [76]:
url = 'https://sec.gov/Archives/edgar/data/1191713/000120919120036662/doc4.xml'
html = urlopen(url)
bsObj = BeautifulSoup(html.read())
bsObj

<?xml version="1.0"?><html><body><ownershipdocument>
<schemaversion>X0306</schemaversion>
<documenttype>4</documenttype>
<periodofreport>2020-06-13</periodofreport>
<notsubjecttosection16>0</notsubjecttosection16>
<issuer>
<issuercik>0001660134</issuercik>
<issuername>Okta, Inc.</issuername>
<issuertradingsymbol>OKTA</issuertradingsymbol>
</issuer>
<reportingowner>
<reportingownerid>
<rptownercik>0001191713</rptownercik>
<rptownername>ARCHAMBEAU SHELLYE L</rptownername>
</reportingownerid>
<reportingowneraddress>
<rptownerstreet1>C/O OKTA, INC. 100 FIRST ST, SUITE 600</rptownerstreet1>
<rptownerstreet2></rptownerstreet2>
<rptownercity>SAN FRANCISCO</rptownercity>
<rptownerstate>CA</rptownerstate>
<rptownerzipcode>94105</rptownerzipcode>
<rptownerstatedescription></rptownerstatedescription>
</reportingowneraddress>
<reportingownerrelationship>
<isdirector>1</isdirector>
<isofficer>0</isofficer>
<istenpercentowner>0</istenpercentowner>
<isother>0</isother>
</reportingownerrelationship>
<

In [100]:
def calculate_owned_pct(x):
    if x.direction == 'D':
        return (-1* x.num_shares/(x.owned_shares + x.num_shares)) * 100
    else:
        return (x.num_shares/(x.owned_shares-x.size+1)) * 100
pd.options.display.max_rows=1000
df['notional_value'] = df['price'] * df['num_shares']
df['transaction%'] = df.apply(calculate_owned_pct, axis=1)
df.sort_values('notional_value', ascending=False)

,ticker,trans_date,price,num_shares,direction,trans_type,person_type,owned_shares,notional_value,transaction%
521,ROAD,2020-06-11,16.0000,2363312,D,S,unknown,1802050,3.781299e+07,-56.737254
520,ROAD,2020-06-11,16.0000,2100000,D,S,unknown,4941831,3.360000e+07,-29.821789
522,ROAD,2020-06-11,16.0000,1286688,D,S,unknown,981119,2.058701e+07,-56.737103
11,ATUS,2020-06-11,26.6400,430000,D,S,unknown,21491295,1.145520e+07,-1.961563
121,ALT,2020-06-15,7.5400,1500000,A,P,major holder,4500000,1.131000e+07,33.333400
233,"BIO, BIOB",2020-06-12,448.8500,21860,D,F,unknown,295122,9.811861e+06,-6.896291
645,KZR,2020-06-11,5.5000,1454545,A,P,major holder,6254545,7.999998e+06,23.255842
1123,PAYC,2020-06-12,302.3920,24337,D,S,director,10020,7.359314e+06,-70.835638
1121,PAYC,2020-06-12,300.5010,23960,D,S,director,53735,7.200004e+06,-30.838535
12,ATUS,2020-06-12,24.1600,290000,D,S,unknown,21201295,7.006400e+06,-1.349384


In [101]:
con = db.create_engine("sqlite:///C:/Users/zheji/Desktop/TradingTools/fundamentalDB.sqlite")
df.to_sql('insiderActivity', con, if_exists='append')

In [46]:
groupby_company = df.groupby('ticker')
for group in df.ticker.unique():
    print(groupby_company.get_group(group))

  ticker        date    price      size direction  owned_shares
0   APRE  2020-06-11  28.7527  235200.0         D     1938549.0
1   APRE  2020-06-11  28.7527    9800.0         D       80772.0
   ticker        date   price     size direction  owned_shares
10   NMTR  2020-06-15  0.5971  10000.0         A       49441.0
   ticker        date  price      size direction  owned_shares
11   ATUS  2020-06-11  26.64  430000.0         D    21491295.0
12   ATUS  2020-06-12  24.16  290000.0         D    21201295.0
   ticker        date  price    size direction  owned_shares
13   CPLG  2020-06-11    0.0  1368.0         A        2863.0
   ticker        date   price      size direction  owned_shares
14   ACRX  2020-06-15  1.3297    7547.0         A      468757.0
15   ACRX  2020-06-15  1.2890  100000.0         D      178750.0
   ticker        date  price     size direction  owned_shares
17    WMS  2020-06-11  15.74   9235.0         A      181862.0
18    WMS  2020-06-11  24.20  23469.0         A      20